In [1]:
import pandas as pd
import numpy as np
import torch


In [2]:
df = pd.read_csv(r'/Users/jamesjirsa/Desktop/Data_Science/JJ_Projects/UFC/new_method/scrape_ufc_stats/final_fight_data.csv')
df

,Unnamed: 0,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,...,delta_ELO_FIGHTER,delta_AGE,delta_REACH,delta_HEIGHT,delta_SIG_STR_vs_avoid,delta_TD_vs_avoid,PC1,PC2,opponent_PC1,opponent_PC2
0,0,UFC25:UltimateJapan3,TitoOrtizvs.WanderleiSilva,W/L,UFC Light Heavyweight Title Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),John McCarthy,...,-45.805984,-1.442847,0.0,-4.0,6.735159,0.000000,-2304.093451,111.659258,118.546726,65.301398
1,1,UFC25:UltimateJapan3,TitoOrtizvs.WanderleiSilva,W/L,UFC Light Heavyweight Title Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),John McCarthy,...,45.805984,1.442847,0.0,4.0,2.471401,-0.099214,118.546726,65.301398,-2304.093451,111.659258
2,4,UFC27:UltimateBadBoyz,JeremyHornvs.EugeneJackson,W/L,Middleweight Bout,Submission,1,4:32,3 Rnd (5-5-5),Mario Yamasaki,...,22.284177,-8.919918,NaN,5.0,-0.250489,0.029975,-849.656402,-362.007419,NaN,NaN
3,5,UFC28:HighStakes,RandyCouturevs.KevinRandleman,W/L,UFC Heavyweight Title Bout,KO/TKO,3,4:13,5 Rnd (5-5-5-5-5),John McCarthy,...,44.264592,8.134155,NaN,3.0,1.838605,0.236066,-1097.109589,-429.197403,NaN,NaN
4,6,UFC28:HighStakes,JensPulvervs.JohnLewis,W/L,Lightweight Bout,KO/TKO,1,0:15,3 Rnd (5-5-5),Mario Yamasaki,...,27.873931,NaN,NaN,-5.0,0.894140,-0.006089,-308.968705,126.665221,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10398,11693,UFCFightNight:Sandhagenvs.Nurmagomedov,ViktoriiaDudakovavs.SamHughes,L/W,Women's Strawweight Bout,Decision - Split,3,5:00,3 Rnd (5-5-5),Daniel Movahedi,...,-53.379668,6.579055,NaN,NaN,-3.026368,0.064299,4453.195258,-934.692022,NaN,NaN
10399,11694,UFCFightNight:Sandhagenvs.Nurmagomedov,SharaMagomedovvs.MichalOleksiejczuk,W/L,Middleweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Daniel Movahedi,...,-24.699481,-0.772074,NaN,-2.0,3.331041,-0.523622,2057.527027,-1488.797293,NaN,NaN
10400,11695,UFCFightNight:Sandhagenvs.Nurmagomedov,TonyFergusonvs.MichaelChiesa,L/W,Welterweight Bout,Submission,1,3:44,3 Rnd (5-5-5),Marc Goddard,...,44.625080,-3.816564,-1.0,2.0,-1.854587,0.117753,8848.342383,-144.344449,13007.510069,-6465.896750
10401,11696,UFCFightNight:Sandhagenvs.Nurmagomedov,MarlonVeravs.DeivesonFigueiredo,L/W,Bantamweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Keith Peterson,...,-57.771347,-4.958248,2.0,3.0,2.894640,-0.048156,15686.349192,-6233.727167,8944.182372,-2843.699104


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10403 entries, 0 to 10402
Columns: 117 entries, Unnamed: 0 to opponent_PC2
dtypes: float64(99), int64(4), object(14)
memory usage: 9.3+ MB


In [4]:
max_fight_idx = df.groupby('FIGHTER')['Fight_Number'].idxmax()
max_fight_rows = df.loc[max_fight_idx]
extra = max_fight_rows[['FIGHTER', 'PC1', 'PC2']]
extra = extra.reset_index(drop=True)
extra

,FIGHTER,PC1,PC2
0,AJDobson,178.871203,-661.693936
1,AJFletcher,-201.858418,-229.327864
2,AaronPhillips,-783.615653,-472.240726
3,AaronRiley,2540.248086,-880.391652
4,AaronRosa,-926.136453,-406.842729
...,...,...,...
1539,ZhalgasZhumagulov,539.977884,-774.510659
1540,ZhangLipeng,349.713406,-113.135222
1541,ZhangTiequan,-2463.923013,180.824915
1542,ZhangWeili,5507.705001,-1225.935803


In [5]:
import plotly.express as px

fig = px.scatter(extra, x='PC1', y='PC2', text='FIGHTER',
                 title='PC1 vs PC2 for Fighters with Maximum Fight Numbers',
                 labels={'PC1': 'Principal Component 1', 'PC2': 'Principal Component 2'})

fig.update_traces(textposition='top center', marker=dict(size=10))
fig.show()


In [6]:
df = df.drop(['PC1', 'PC2'], axis = 1)

In [7]:
df['RESULT'] = np.where(df['RESULT'] == 'Win', 1,0)

In [8]:
features = df.columns[df.columns.get_loc('ELO_FIGHTER')+1:].tolist()
#features.remove('CUM_index')
features.remove('DOB')
df = df.dropna(subset = 'opponent_PC1')
#df = df.drop(['CUM_index'], axis = 1)

In [9]:
X_train = df[(df['DATE'] >= '2005') & (df['DATE'] < '2024')]
y_train = X_train['RESULT']
X_train = X_train[features]
X_test = df[(df['DATE'] >= '2024')]
y_test = X_test['RESULT']
X_test = X_test[features]
X_train

,CUM_KD,CUM_SIG.STR._Landed,CUM_SIG.STR._Attempted,CUM_SIG.STR._Attempted_avoided,CUM_TOTAL STR._Landed,CUM_TOTAL STR._Attempted,CUM_TOTAL STR._Attempted_avoided,CUM_TD_Landed,CUM_TD_Attempted,CUM_TD_Attempted_avoided,...,delta_CUM_GROUND_Landed_PM,delta_CUM_SUB.ATT_PM,delta_ELO_FIGHTER,delta_AGE,delta_REACH,delta_HEIGHT,delta_SIG_STR_vs_avoid,delta_TD_vs_avoid,opponent_PC1,opponent_PC2
127,4.0,125.0,254.0,52.0,149.0,280.0,52.0,1.0,2.0,7.0,...,-0.595179,0.059553,9.252077,-2.918549,-3.0,-5.0,-1.039820,-0.208319,-1686.440735,-33.892203
128,1.0,85.0,205.0,30.0,181.0,311.0,31.0,3.0,5.0,0.0,...,-0.223441,-0.016748,-29.694413,8.027379,-4.0,1.0,0.040092,0.019479,-404.099894,-128.585700
130,0.0,41.0,108.0,74.0,90.0,165.0,77.0,12.0,21.0,2.0,...,0.223441,0.016748,29.694413,-8.027379,4.0,-1.0,0.425290,0.354331,-245.417718,-477.552826
131,2.0,162.0,278.0,60.0,182.0,299.0,60.0,1.0,1.0,3.0,...,0.595179,-0.059553,-9.252077,2.918549,3.0,5.0,11.308756,-0.129072,-596.172258,-603.930910
132,1.0,399.0,713.0,121.0,1108.0,1457.0,135.0,38.0,58.0,8.0,...,0.780805,-0.085291,40.492379,2.187543,0.0,3.0,2.198596,0.222327,172.349450,-874.033760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10021,1.0,179.0,303.0,25.0,288.0,421.0,28.0,4.0,12.0,3.0,...,0.468798,0.099379,11.580876,-11.696099,2.0,1.0,2.335976,0.067498,13947.713613,-6190.022887
10022,10.0,1186.0,2587.0,596.0,1398.0,2825.0,662.0,5.0,12.0,9.0,...,-0.468798,-0.099379,-11.580876,11.696099,-2.0,-1.0,3.580086,-0.056823,212.659292,133.762182
10023,11.0,729.0,2002.0,785.0,789.0,2086.0,794.0,11.0,29.0,5.0,...,-0.647416,-0.101734,-8.354983,9.585216,0.0,-4.0,2.519132,0.032150,4499.653023,449.632336
10024,1.0,130.0,281.0,77.0,179.0,336.0,81.0,2.0,8.0,9.0,...,-0.090649,0.097999,10.047904,-10.891170,-3.0,-1.0,0.479150,-0.034376,12162.340109,-6077.626607


In [10]:
X_train = df[(df['DATE'] >= '2005') & (df['DATE'] < '2024')]
y_train = X_train['RESULT']
X_train = X_train[features]
X_test = df[(df['DATE'] >= '2024')]
y_test = X_test['RESULT']
X_test = X_test[features]

X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values



X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


In [11]:
X_train

tensor([[ 4.0000e+00,  1.2500e+02,  2.5400e+02,  ..., -2.0832e-01,
         -1.6864e+03, -3.3892e+01],
        [ 1.0000e+00,  8.5000e+01,  2.0500e+02,  ...,  1.9479e-02,
         -4.0410e+02, -1.2859e+02],
        [ 0.0000e+00,  4.1000e+01,  1.0800e+02,  ...,  3.5433e-01,
         -2.4542e+02, -4.7755e+02],
        ...,
        [ 1.1000e+01,  7.2900e+02,  2.0020e+03,  ...,  3.2150e-02,
          4.4997e+03,  4.4963e+02],
        [ 1.0000e+00,  1.3000e+02,  2.8100e+02,  ..., -3.4376e-02,
          1.2162e+04, -6.0776e+03],
        [ 0.0000e+00,  1.5800e+02,  3.5700e+02,  ...,  1.9214e-01,
          3.0868e+03,  4.4348e+02]])

In [12]:
import pyro
import pyro.distributions as dist
import torch.nn as nn
import pyro.infer
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
import pyro.nn as pnn

class BayesianNN(pnn.PyroModule):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = pnn.PyroModule[nn.Linear](input_dim, hidden_dim)
        self.fc1.weight = pnn.PyroSample(dist.Normal(0., 1.).expand([hidden_dim, input_dim]).to_event(2))
        self.fc1.bias = pnn.PyroSample(dist.Normal(0., 1.).expand([hidden_dim]).to_event(1))

        self.fc2 = pnn.PyroModule[nn.Linear](hidden_dim, hidden_dim)
        self.fc2.weight = pnn.PyroSample(dist.Normal(0., 1.).expand([hidden_dim, hidden_dim]).to_event(2))
        self.fc2.bias = pnn.PyroSample(dist.Normal(0., 1.).expand([hidden_dim]).to_event(1))

        self.fc3 = pnn.PyroModule[nn.Linear](hidden_dim, output_dim)
        self.fc3.weight = pnn.PyroSample(dist.Normal(0., 1.).expand([output_dim, hidden_dim]).to_event(2))
        self.fc3.bias = pnn.PyroSample(dist.Normal(0., 1.).expand([output_dim]).to_event(1))

        self.relu = nn.ReLU()

    def forward(self, x, y=None):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        logits = self.fc3(x).squeeze(-1)
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Bernoulli(logits=logits), obs=y)
        return logits




In [13]:
from pyro.infer.autoguide import AutoDiagonalNormal

# Instantiate the model
input_dim = X_train.shape[1]
hidden_dim = 100
output_dim = 1

model = BayesianNN(input_dim, hidden_dim, output_dim)

# Define the autoguide
guide = AutoDiagonalNormal(model)



In [14]:
adam = Adam({"lr": 0.001})  # Experiment with a smaller learning rate

# Define the loss function
criterion = nn.BCEWithLogitsLoss()

# Setup the inference algorithm with the autoguide
svi = SVI(model, guide, adam, loss=Trace_ELBO())

# Training loop
num_iterations = 50000
for j in range(num_iterations):
    loss = svi.step(X_train, y_train)
    if j % 100 == 0:
        print(f"Step {j} : loss = {loss}")



Step 0 : loss = 25945353.610539854
Step 100 : loss = 28008437.84683633
Step 200 : loss = 14029137.79962778
Step 300 : loss = 11809551.143677592
Step 400 : loss = 9492450.027695358
Step 500 : loss = 23032581.158735573
Step 600 : loss = 11592760.093347847
Step 700 : loss = 10541785.863761902
Step 800 : loss = 10380483.834409118
Step 900 : loss = 6833656.0522789955
Step 1000 : loss = 3510290.441319883
Step 1100 : loss = 4156418.546306908
Step 1200 : loss = 2979766.2300432324
Step 1300 : loss = 2410468.2464383245
Step 1400 : loss = 2108479.241768062
Step 1500 : loss = 2145432.3581269383
Step 1600 : loss = 888457.5626139641
Step 1700 : loss = 468343.07884544134
Step 1800 : loss = 356976.4158305526
Step 1900 : loss = 428755.89092993736
Step 2000 : loss = 80869.45470023155
Step 2100 : loss = 185449.56737709045
Step 2200 : loss = 493440.84857809544
Step 2300 : loss = 72331.62661373615
Step 2400 : loss = 73628.2122361064
Step 2500 : loss = 80824.26822710037
Step 2600 : loss = 259361.8061403036


In [15]:
import pyro.poutine as poutine

def predict(model, guide, X):
    sampled_models = []
    for _ in range(100):
        # Sample parameters using the guide
        sampled_params = guide(X)
        
        # Use poutine to substitute sampled parameters into the model
        sampled_model = poutine.condition(model, sampled_params)
        
        # Make a prediction with the sampled model
        with torch.no_grad():
            logits = sampled_model(X)
            sampled_models.append(logits.sigmoid())
    
    yhats = torch.stack(sampled_models).mean(0)
    return yhats

y_pred = predict(model, guide, X_test)
accuracy = ((y_pred > 0.5).float() == y_test).float().mean()
print(f"Accuracy: {accuracy.item():.4f}")



Accuracy: 0.5000


In [16]:
import pyro.poutine as poutine

def uncertainty_estimate(model, guide, X):
    sampled_models = []
    for _ in range(1000):
        # Sample parameters using the guide
        sampled_params = guide(X)
        
        # Use poutine to substitute sampled parameters into the model
        conditioned_model = poutine.condition(model, sampled_params)
        
        # Make a prediction with the conditioned model
        with torch.no_grad():
            logits = conditioned_model(X)
            sampled_models.append(logits.sigmoid())
    
    yhats = torch.stack(sampled_models)
    mean_prediction = yhats.mean(0)
    uncertainty = yhats.std(0)
    return mean_prediction, uncertainty

mean_pred, uncertainty = uncertainty_estimate(model, guide, X_test)

 

In [17]:
mean_pred

tensor([0.4997, 0.5007, 0.4997, 0.4997, 0.4997, 0.4997, 0.4997, 0.4992, 0.4997,
        0.4997, 0.4997, 0.5002, 0.4997, 0.4997, 0.4992, 0.4992, 0.4992, 0.5007,
        0.4992, 0.4997, 0.5007, 0.4997, 0.4997, 0.4997, 0.4997, 0.4997, 0.4997,
        0.4992, 0.4997, 0.4997, 0.4992, 0.4997, 0.4997, 0.4997, 0.4997, 0.4992,
        0.4997, 0.4997, 0.4992, 0.4997, 0.4997, 0.4997, 0.4997, 0.5012, 0.4992,
        0.5002, 0.5012, 0.4997, 0.4997, 0.4992, 0.4992, 0.4997, 0.4997, 0.4997,
        0.4997, 0.5007, 0.4997, 0.4997, 0.4997, 0.4992, 0.4992, 0.4997, 0.5007,
        0.4997, 0.4993, 0.4997, 0.4992, 0.4997, 0.4997, 0.4987, 0.4997, 0.4997,
        0.4997, 0.4997, 0.5012, 0.4992, 0.4992, 0.4997, 0.4997, 0.4997, 0.4997,
        0.4992, 0.5012, 0.4997, 0.4995, 0.4997, 0.4997, 0.4997, 0.4997, 0.4992,
        0.4997, 0.5007, 0.4997, 0.4997, 0.4997, 0.4997, 0.4997, 0.4997, 0.4997,
        0.4992, 0.4997, 0.4997, 0.4997, 0.4997, 0.4992, 0.4997, 0.4992, 0.4997,
        0.4997, 0.4997, 0.4997, 0.4997, 

In [18]:
uncertainty

tensor([0.0167, 0.0279, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0053, 0.0167,
        0.0167, 0.0167, 0.0321, 0.0167, 0.0167, 0.0053, 0.0053, 0.0053, 0.0279,
        0.0053, 0.0167, 0.0422, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167,
        0.0053, 0.0167, 0.0167, 0.0053, 0.0167, 0.0167, 0.0167, 0.0167, 0.0053,
        0.0167, 0.0167, 0.0053, 0.0167, 0.0167, 0.0167, 0.0167, 0.0321, 0.0053,
        0.0230, 0.0321, 0.0167, 0.0167, 0.0053, 0.0053, 0.0167, 0.0167, 0.0167,
        0.0167, 0.0279, 0.0167, 0.0167, 0.0167, 0.0053, 0.0053, 0.0167, 0.0279,
        0.0167, 0.0192, 0.0167, 0.0053, 0.0167, 0.0167, 0.0166, 0.0167, 0.0167,
        0.0167, 0.0167, 0.0321, 0.0053, 0.0053, 0.0167, 0.0167, 0.0167, 0.0167,
        0.0053, 0.0321, 0.0167, 0.0114, 0.0167, 0.0167, 0.0167, 0.0167, 0.0053,
        0.0167, 0.0279, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167,
        0.0053, 0.0167, 0.0167, 0.0167, 0.0167, 0.0053, 0.0167, 0.0053, 0.0167,
        0.0167, 0.0167, 0.0167, 0.0167, 